In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import re

# https://en.wikipedia.org/wiki/Global_Industry_Classification_Standard 

In [6]:
def process_df(x):
    columns = x.iloc[0,:]
    columns.name = ""
    x.columns = columns
    x = x.iloc[1:,:]
    x.reset_index(drop=True, names=columns, inplace=True)
    x.rename(index={"6":""}).head(7)
    return x


def get_working_capital(zone=""):
    path = "data_damodaran/cash_flow_estimation/working_capital/wcdata" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_growth_historical(zone=""):
    path = "data_damodaran/growth/historical_growth/histgr" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x

def get_capex(zone=""):
    path = "data_damodaran/cash_flow_estimation/CAPEX/capex" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x


def get_taxrates(zone=""):
    path = "data_damodaran/discount_rate_estimation/tax/tax_rate_country/taxrate" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry name")
    return x


def get_growth_ebit(zone=""):
    path = "data_damodaran/growth/growth_ebit/fundgrEB" + zone + ".xls"
    x = pd.read_excel(path, sheet_name=None)
    x = x["Industry Averages"]
    loc = x[x.iloc[:,0] == "Advertising"].index.values[0] - 1
    x = x.iloc[loc:,:]
    x = process_df(x)
    x = x.set_index("Industry Name")
    return x




In [47]:
%%time
n = 4
x = yf.Ticker("MSFT")
sector = "Software (System & Application)"
x.info#.get('industry')

CPU times: user 36.6 ms, sys: 4.75 ms, total: 41.4 ms
Wall time: 183 ms


{'address1': 'One Microsoft Way',
 'city': 'Redmond',
 'state': 'WA',
 'zip': '98052-6399',
 'country': 'United States',
 'phone': '425 882 8080',
 'website': 'https://www.microsoft.com',
 'industry': 'Software - Infrastructure',
 'industryKey': 'software-infrastructure',
 'industryDisp': 'Software - Infrastructure',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solutions, including Dynamics 365, a set of intelligent, cloud-based applications across ERP, CRM, power 

In [16]:
def freecashflow(ticket, sector,n=4):
    x = yf.Ticker(ticket)
    ebit = x.income_stmt.loc["EBIT",:].sort_index()
    g_ebit = get_growth_ebit("").loc[sector,"Expected Growth in EBIT"]
    ebit_forecast = pd.Series([ebit.iloc[-1]*((1+g_ebit)**i) for i in range(n+1)], index=range(0,n+1))[1:]

    tax_rate = get_taxrates("").loc[sector, "Average across only money-making companies"]

    usa_ktn_percentage = get_working_capital("").loc[sector,"Non-cash WC/ Sales"]
    sales_growth_2y = get_growth_historical("").loc[sector,"Expected Growth in Revenues - Next 2 years"]
    g_sales = (1+sales_growth_2y)**(1/2)-1

    sales = x.incomestmt.loc["Total Revenue",:].sort_index()
    sales_forecast = pd.Series([sales.iloc[-1]*((1+g_sales)**i) for i in range(n+1)], index=range(0,n+1))
    sales_forecast

    ktn_forecast = sales_forecast * usa_ktn_percentage
    ktn_delta_forecast = ktn_forecast.diff()[1:]

    capex_vs_sales = get_capex("").loc[sector, "Net Cap Ex/Sales"]
    capex_net = (sales_forecast * capex_vs_sales)[1:]


    return ebit_forecast*(1-tax_rate) - capex_net - ktn_delta_forecast

In [44]:
freecashflow("MSFT", "Software (System & Application)")

yf.Ticker("KO").info.get("industry")

list(get_capex("").index)
yf.Ticker("KO").info.get("industry")

freecashflow("META", "Beverage (Soft)")

yf.Ticker("META").info.get("industry")

'Internet Content & Information'

1   NaN
2   NaN
3   NaN
4   NaN
dtype: float64